# <center>Welcome FSP solved with diverse heuristics</center>

## Data Utils

In [12]:
import numpy as np
import itertools
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from utils.benchmarks import benchmarks

In [11]:


'''
Reading the data from the benchmark file
'''

def read_flow_shop_data(file_path, machine_count, job_count):
    instances = []
    with open(file_path) as p:
        lines = p.readlines()
        line_count = len(lines)

        instance_count = line_count // (machine_count + 3)

        for i in range(instance_count):
            # recover the data of each instance
            params_line = lines[i * (machine_count + 3) + 1]
            job_count, machine_count, initial_seed, upper_bound, lower_bound = list(
                map(lambda x: int(x), params_line.split()))

            # processing_times = [list(map(int, lines[i * (machine_count + 3) + 3])) for line in lines]
            processing_times = np.array([list(map(lambda x: int(x), line.strip().split())) for
                                         line in lines[
                                                 i * (machine_count + 3) + 3:  # start
                                                 i * (machine_count + 3) + 3 + machine_count  # end
                                                 ]
                                         ])

            record = (machine_count, job_count, processing_times)
            instances.append(record)

    return instances



### Path Cost calculation function :
used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [8]:
def incremental_cost(machine_job_matrix, jobs_sequence):
    nb_jobs = len(jobs_sequence)
    nb_machines = machine_job_matrix.shape[0]

    incremental_cost = np.zeros((nb_machines, nb_jobs))

    # evaluate the first machines
    incremental_cost[0, 0] = machine_job_matrix[0][jobs_sequence[0]]

    for i in range(1, nb_jobs):
        incremental_cost[0][i] = incremental_cost[0][i - 1] + \
            machine_job_matrix[0][jobs_sequence[i]]

    # evaluate the rest of machines
    for i in range(1, nb_machines):
        incremental_cost[i, 0] = incremental_cost[i - 1, 0] + \
            machine_job_matrix[i, jobs_sequence[0]]
        for j in range(1, nb_jobs):
            incremental_cost[i, j] = machine_job_matrix[i, jobs_sequence[j]] + \
                max(incremental_cost[i - 1, j], incremental_cost[i, j - 1])
    return incremental_cost[-1,-1]

### Gantt graph generator

In [4]:
def generate_gantt_chart(current_instance, solution):
    plt.figure(figsize=(20, 12))
    df = pd.DataFrame(columns=['Machine', 'Job', 'Start', 'Finish'])

    machines, jobs = current_instance.shape
    machine_times = np.zeros((machines, jobs))
    start_time_m = np.zeros(machines)
    for job in solution:

        for machine_index in range(machines):
            start_time = start_time_m[machine_index]
            if machine_index > 0:
                start_time = max(start_time, start_time_m[machine_index-1])
            end_time = start_time + \
                current_instance[machine_index, job]
            start_time_m[machine_index] = end_time

            df = pd.concat([df, pd.DataFrame({'Machine': f'Machine {machine_index + 1}',
                                                'Job': f'Job {job + 1}',
                                                'Start': start_time,
                                                'Finish': end_time}, index=[0])], ignore_index=True)

            machine_times[machine_index, job] = end_time

    colors = plt.cm.tab10.colors
    for i, machine_index in enumerate(range(machines)):
        machine_df = df[df['Machine'] == f'Machine {machine_index + 1}']
        plt.broken_barh([(start, end - start) for start, end in zip(machine_df['Start'], machine_df['Finish'])],
                        (i * 10, 9), facecolors=[colors[j % 10] for j in range(jobs)], edgecolor='black')

    plt.xlabel('Time')
    plt.yticks([i * 10 + 4.5 for i in range(machines)],
                [f'Machine {i + 1}' for i in range(machines)])
    plt.show()

## Heuristics

### Johnson **n** jobs **2** machines

In [5]:
def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        
    return m1+m2

### Test Johnson

In [6]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(7,2), low=5, high=23)
print(rnd_data, "\n")

start_time = time.time()
sol = johnson_method(rnd_data)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Johnson is {sol} with a makespan of {incremental_cost(np.array(rnd_data).T,sol)}')
print("Elapsed time:", elapsed_time, "seconds")

[[18  8]
 [20 10]
 [19 17]
 [ 8 13]
 [21  6]
 [13 10]
 [21 12]] 

Best sequence found by Johnson is [3, 2, 6, 5, 1, 0, 4] with a makespan of 126.0
Elapsed time: 0.0 seconds


### CDS Heuristic [Campbell,Dudek and Smith ] : **n** jobs **m** machines (1970)

In [7]:
# python code for CDS heuristic

def cds_heuristic(processing_times):
    
    
    nb_machines, nb_jobs = processing_times.shape


    best_cost = math.inf

    
    machine_1_times = np.zeros((nb_jobs,1))
    machine_2_times = np.zeros((nb_jobs,1))
    
    
    # iterate through the nb_machines-1 auxiliary n-job 2-machines problems



    for k in range(nb_machines -1):
        machine_1_times[:,0] += processing_times[:][k]
        machine_2_times[:,0] += processing_times[:][-k-1]
        
        jn_times = np.concatenate((machine_1_times, machine_2_times), axis=1)
        seq = johnson_method(jn_times)
        cost = incremental_cost(jn_times.T,seq)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
    
    return best_seq, best_cost


### Test CDS

In [8]:
rnd_data = np.random.randint(size=(8,5), low=10, high=50)
print(rnd_data, "\n")
cds_heuristic(rnd_data.T)

[[34 18 40 26 12]
 [35 37 23 39 25]
 [14 37 40 33 25]
 [45 18 14 14 42]
 [17 27 11 44 33]
 [36 19 32 24 26]
 [30 36 49 23 22]
 [20 20 24 11 21]] 



([2, 4, 7, 3, 5, 1, 6, 0], 243.0)

In [9]:
instances = read_flow_shop_data('../Lab1/data/tai20_5.txt', 5, 20)
comman_instance = instances[0][2]

In [10]:
start_time = time.time()
best_solution,best_cost  = cds_heuristic(comman_instance)


best_cost = incremental_cost(comman_instance,best_solution)

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of CDS:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')



Results of CDS:
Best sequence is [14, 2, 8, 13, 16, 7, 6, 0, 18, 3, 10, 15, 11, 1, 4, 5, 19, 17, 9, 12] with a makespan of 1390.0.
Elapsed time of 0.003493785858154297 seconds.


### NEH Algorithm

In [11]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [12]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [13]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if incremental_cost(processing_times.T,[J1, J2]) < incremental_cost(processing_times.T,[J2, J1]) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = incremental_cost(processing_times.T,new_sequence)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

In [14]:
start_time = time.time()
best_solution,best_cost  = neh_algorithm(instances[6][2].T)

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of NEH:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')


Results of NEH:
Best sequence is [4, 2, 19, 10, 7, 5, 3, 8, 1, 12, 6, 18, 16, 9, 14, 15, 0, 17, 13, 11] with a makespan of 1284.0.
Elapsed time of 0.04602694511413574 seconds.


### HAM Heuristic

In [16]:
def incremental_cost(processing_times, sequence):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [17]:
def indices_from_list_values(master: np.ndarray, values: list) -> list:
    indexes = []
    for element in values:
        for i in range(master.size):
            if master[i] == element and i not in indexes:
                indexes.append(i)
                continue
    return indexes

In [18]:
def ham_sol_1(Pi1: np.ndarray, Pi2: np.ndarray) -> list:
    diff = Pi2 - Pi1
    sol = np.argsort(diff, axis=0).tolist()
    sol.reverse()       # in decreasing order
    return sol

In [19]:
def ham_sol_2(Pi1: np.ndarray, Pi2: np.ndarray) -> list:

    diff = Pi2 - Pi1
    according_pi1 = np.argwhere(diff >= 0)
    according_pi2 = np.argwhere(diff < 0)

    # Order Pi1 in increasing order
    Pi1_sorted= np.sort(Pi1[according_pi1], axis=None).tolist()
    Pi1_list = indices_from_list_values(Pi1, Pi1_sorted)

    # Order Pi2 in decreasing order
    Pi2_sorted = np.sort(Pi2[according_pi2], axis=None).tolist()
    Pi2_sorted.reverse()
    Pi2_list = indices_from_list_values(Pi2, Pi2_sorted)

    return Pi1_list + Pi2_list

In [20]:
def ham_heuristic(processing_times: np.ndarray) -> list:
    _ , m = processing_times.shape
    left = processing_times[:, :int(m/2)]
    right = processing_times[:, int(m/2):]

    Pi1 = left.sum(axis=1)
    Pi2 = right.sum(axis=1)

    solution1 = ham_sol_1(Pi1, Pi2)
    solution2 = ham_sol_2(Pi1, Pi2)
    Cmax1 = incremental_cost(processing_times, solution1)
    Cmax2 = incremental_cost(processing_times, solution2)
    
    if Cmax1 < Cmax2:
        return solution1, Cmax1
    else:
        return solution2, Cmax2

### TEST HAM

In [21]:
start_time = time.time()
best_solution, best_cost  = ham_heuristic(benchmarks[0])

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of HAM:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of HAM:
Best sequence is [2, 8, 16, 14, 18, 10, 1, 12, 15, 7, 13, 5, 0, 4, 9, 17, 3, 6, 19, 11] with a makespan of 1417.0.
Elapsed time of 0.0015211105346679688 seconds.


### PALMER Heuristic

In [22]:
class PalmerHeuristic:
    def __init__(self, dist_mat):
        self.dist_mat = dist_mat
        self.c = None
        self.ordre_opt = []
        self.nb_machines = dist_mat.shape[0]
        self.nb_jobs = dist_mat.shape[1]
        self.weights = None
 

    def init_weights(self):      #middle machine has a weight of 0 and it increases/decreases toward last/first machine at same rate (Example for 3 machines: the weights are w1 =-2 w2 =0 w3 =+2)
        lst = np.array([ (2*i - 1 - self.nb_machines) for i in range(self.nb_machines)])
        return lst - np.mean(lst)
   

    def compute_weighted_sum(self):       #Computing weighted sum for each job (Example for j1: w1*d11 + w2*d12 + w3*d13)
        weighted_sum = []
        for i in range(self.nb_jobs):
            somme = np.dot(self.dist_mat[:,i],self.weights)
            weighted_sum.append(somme)
        return np.array(weighted_sum)
   

    def sum_per_machine(self):      #Find the sum of time of each machine
        return self.dist_mat.sum(axis=1)
                                 
    
    def update_c(self, c, ordre):         # Update the computation time to get the makespan
        c[0][0] = self.dist_mat[0,ordre[0]]
        for j in range(1,self.nb_machines):
            c[0][j] = self.dist_mat[j,ordre[0]] + c[0][j-1]
        for j in range (1, self.nb_jobs):
            c[j][0] = self.dist_mat[0, ordre[j]] + c[j-1][0]
        for i in range(1, self.nb_jobs):
            for j in range(1,self.nb_machines):
                c[i][j] = max([c[i-1][j],c[i][j-1]]) + self.dist_mat[j,ordre[i]]
        return c 

    
    def get_cmax(self):               # Cmax : makespan
        return self.c[len(self.ordre_opt)-1][self.dist_mat.shape[0]-1]
    
    
    def lower_bound(self):                     #Define the lower bound (upperbound is the cmax found)
        lb = []
        for i in range(self.nb_machines):
            a = self.sum_per_machine()
            if i == 0:
                bound = a[i] + min(self.dist_mat.T[:,1:].sum(axis=1))
            
            elif i == (self.nb_machines -1) :
                bound = a[i] + min(self.dist_mat.T[:,:i].sum(axis=1)) 
            
            else :
                bound = a[i] + min(self.dist_mat.T[:,:i].sum(axis=1)) + min(self.dist_mat.T[:,i+1])
            
            lb.append(bound)
        return max(lb)
    

    def run(self):
        self.weights = self.init_weights()
        C = np.zeros((self.nb_jobs, self.nb_machines))
        
        self.c = C
        a = self.compute_weighted_sum()
        self.ordre_opt = np.argsort(a)[::-1] #argsort effectue le tri dans l'ordre ascendant donc on l'inverse
    
        self.update_c(self.c,self.ordre_opt)
        lb = self.lower_bound()
        print('optimal order for Palmer')
        print(self.ordre_opt + 1)
        print('\nCmax: ' + str(self.get_cmax()))
        print('Lower Bound: ' + str(lb))
        print(self.c)        

### PALMER Test

In [23]:
a = PalmerHeuristic(instances[0][2])
a.run()

optimal order for Palmer
[ 9 11 17 15 16 19  3  6 14  8  2  4  1  5 13  7 12 10 18 20]

Cmax: 1384.0
Lower Bound: 1232
[[  27.   32.   89.  138.  207.]
 [ 103.  106.  113.  223.  309.]
 [ 135.  156.  182.  277.  367.]
 [ 147.  203.  266.  333.  414.]
 [ 224.  238.  313.  373.  501.]
 [ 292.  297.  390.  441.  569.]
 [ 307.  318.  439.  472.  589.]
 [ 343.  413.  484.  575.  624.]
 [ 372.  488.  529.  616.  673.]
 [ 410.  548.  571.  675.  716.]
 [ 493.  551.  660.  733.  789.]
 [ 564.  663.  678.  801.  886.]
 [ 618.  742.  758.  867.  944.]
 [ 695.  798.  887.  965. 1018.]
 [ 709.  871.  950. 1004. 1026.]
 [ 762.  970. 1030. 1043. 1096.]
 [ 853. 1031. 1032. 1052. 1168.]
 [ 940. 1087. 1151. 1236. 1249.]
 [1027. 1173. 1248. 1325. 1343.]
 [1121. 1250. 1290. 1356. 1384.]]


### PRSKE Heuristic

In the new PRSKE priority rule, all jobs are ordered by the non-increasing sum of : $$ AVG_i + STD_i + |SKE_i|$$ 
where:

- $ AVG_i $ represents the average processing time of job _i_.
- $ STD_i $ stands for the standard deviation of job _i_.
- $ |SKE_i| $ denotes the absolute value of the skewness of job _i_.

The parameters are defined as follows:

- $$ AVG_i = \frac{1}{m} \sum_{k=1}^{m} t_{i,k}$$
- $$ STD_i = \sqrt{\frac{1}{m-1} \sum_{k=1}^{m} (t_{i,k} - AVG_i)^2} $$
- $$ SKE_i = \frac{\frac{1}{m} \sum_{k=1}^{m} (t_{i,k} - AVG_i)^3}{\left({\sqrt{\frac{1}{m} \sum_{k=1}^{m} (t_{i,k} - AVG_i)^2}}\right)^3} $$

where:

- $ t_{i,k} $ represents the processing time of job _i_ on machine _k_.
- $ m $ is the number of machines.



In [24]:
def AVG(processing_times):
    return np.mean(processing_times, axis=1)

In [25]:
def STD(processing_times):
    return np.std(processing_times, axis=1, ddof=1)

In [26]:
def skewness_SKE(processing_times):
    """
    Calculates the skewness of job processing times across machines.
    Return : Skewness values for each job
    
    """
    num_jobs, num_machines = processing_times.shape
    skewness_values = []

    
    for i in range(num_jobs):
        avg_processing_time = np.mean(processing_times[i,:])
        numerateur = 0
        denominateur = 0

        for j in range(num_machines):
            som = (processing_times[i,j] - avg_processing_time)
            numerateur += som ** 3
            denominateur += som ** 2

        numerateur *= (1 / num_machines)
        denominateur = (np.sqrt(denominateur * (1 / num_machines))) ** 3

        skewness_values.append(numerateur / denominateur)
        
    return np.array(skewness_values)

In [27]:
def PRSKE(processing_times):
    """
    Calculates the job sequence based on the PRSKE priority rule

    """
    avg = AVG(processing_times)   # Calculate average processing times

    std = STD(processing_times)   #Calculate standard deviation processing times

    skw = np.abs(skewness_SKE(processing_times)) # Calculate Skewness 

    order = skw + std + avg  

    # Sort in descending order
    sorted_order = sorted(zip(order, list(range(processing_times.shape[0]))),reverse=True)

    sequence = [job for _ , job in sorted_order]

    incremental_cost_value = incremental_cost(processing_times, sequence)  # Calculte incremental_cost

    return sequence,  incremental_cost_value 

### TEST PRSKE

In [28]:
start_time = time.time()
best_solution, best_cost  = PRSKE(benchmarks[0])

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of PRSKE:')
print(f'Best sequence is {best_solution} \n with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of PRSKE:
Best sequence is [3, 17, 10, 1, 9, 11, 4, 6, 19, 18, 15, 5, 0, 12, 8, 14, 13, 7, 16, 2] 
 with a makespan of 1593.0.
Elapsed time of 0.004652738571166992 seconds.


### CHEN Heuristic (1983)

Chen (1983) propose une heuristique qui consiste à procéder de la manière suivante :

- on calcule la somme de temps opératoires $ S(i) $  pour chaque tâche i avec :
$$ S(i) = \sum_{k=1}^{m} t_{i,k} $$
- on trouve la tâche c qui a la $ S(i) $ la plus importante et on enlève cette tâche de l'ensemble de tâches non ordonnancées.
- on ordonnance les tâches qui ont $ P_{i1} <= P_{im} $ dans l'ordre croissant de $ P_{i1} $ et on obtient un ordonnancement partiel $ SA $.
- on ordonnance les tâches qui ont $ P_{i1}>P_{im} $ dans l'ordre décroissant de $ P_{im} $ et on obtient un ordonnancement partiel $ SB $ .
- on retient comme solution finale l'ordonnancement $ (SA, c, SB) $

In [29]:
def chen_heuristic(processing_times):

    num_jobs = len(processing_times)
    
    sum_processing_times = [sum(processing_times[i]) for i in range(num_jobs)] # Calcule de la somme de temps opératoires S(i) pour chaque tâche i

    job_max_sum = max(sum_processing_times)
    job_c = sum_processing_times.index(job_max_sum)
    
    remaining_jobs = [i for i in range(num_jobs) if i != job_c]
   
    
    sorted_jobs_le = sorted(remaining_jobs, key=lambda i: processing_times[i][0])
    
    sorted_jobs_gt = sorted(remaining_jobs, key=lambda i: processing_times[i][-1], reverse=True)
   
    S_a = [i for i in sorted_jobs_le if processing_times[i][0] <= processing_times[i][-1]]
    S_b = [i for i in sorted_jobs_gt if processing_times[i][0] > processing_times[i][-1]]
    
    sequence = S_a + [job_c] + S_b
    
    incremental_cost_value = incremental_cost(processing_times, sequence)
    
    return sequence, incremental_cost_value

### Test Chen heuristic

In [30]:
start_time = time.time()

best_solution, best_cost  = chen_heuristic(benchmarks[0])

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of CHEN:')
print(f'Best sequence is {best_solution} \n with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of CHEN:
Best sequence is [14, 2, 8, 13, 16, 7, 6, 0, 18, 3, 10, 15, 4, 11, 1, 5, 19, 17, 9, 12] 
 with a makespan of 1390.0.
Elapsed time of 0.0 seconds.


### GUPTA's Method

In [2]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]
    
    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

In [3]:
def min_sum_processing(job_index, processing_times):
    min_sum = np.inf
    for i in range(processing_times.shape[1] - 1):
        sum_for_pair = processing_times[job_index,
                                        i] + processing_times[job_index, i + 1]
        if sum_for_pair < min_sum:
            min_sum = sum_for_pair
    return min_sum

In [4]:
def calculate_priority(job_index, processing_times):
    diff = float(processing_times[job_index, 0] -
                 processing_times[job_index, -1])
    sign = (diff > 0) - (diff < 0)
    return sign / min_sum_processing(job_index, processing_times)

In [5]:
def gupta_heuristic(processing_times):
    priorities = [calculate_priority(i, processing_times)
                  for i in range(processing_times.shape[0])]
    total_times = [np.sum(processing_times[i])
                   for i in range(processing_times.shape[0])]
    sequence = sorted(range(len(priorities)), key=lambda k: (priorities[k], total_times[k]))
    best_makespan = calculate_makespan(processing_times,sequence)
    return sequence, best_makespan

### GUPTA TEST

In [13]:
start_time = time.time()

best_solution, best_cost  = gupta_heuristic(benchmarks[0])

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of GUPTA:')
print(f'Best sequence is {best_solution} \n with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of GUPTA:
Best sequence is [10, 2, 8, 16, 14, 15, 7, 13, 0, 3, 18, 6, 4, 5, 9, 17, 1, 19, 12, 11] 
 with a makespan of 1396.
Elapsed time of 0.0009999275207519531 seconds.
